In [1]:
import json
import sys
from zipfile import ZipFile

import os

import datetime
import pytz

from log import TelegramLog, PrintLog

from pprint import pprint
from translator import Translator
from sunless import RecursiveUpdateProcessor, diff_tree

OUTPUT_PATH = 'output'
ORIGIN_PATH = 'entities'
TRANS_PATH = 'entities_ko'
STATUS_PATH = 'status'

CONFIG_FILE = 'config.json'

# Load configure

In [2]:
from config import config

# Initialize Telegram

In [3]:
if config['logMode'] == 'telegram':
    log = TelegramLog(config['botToken'], config['botGroupId'])
else:
    log = PrintLog()
    
log.log("""오늘도 좋은 하루! 썬리스 씨봇입니다! 
😛 오늘 버전 🇰🇷 패치 제작을 시작합니다! 우우우웅~""") 

오늘도 좋은 하루! 썬리스 씨봇입니다! 
😛 오늘 버전 🇰🇷 패치 제작을 시작합니다! 우우우웅~


# Initialize Google API

In [4]:
print("Initialize Google API")
translator = Translator(config['key'], config['sheetId'])
translator.load_noun()

Initialize Google API


# Load translations

In [7]:
events = {}
print("Loading events sheet 1/3")
event1_trans = translator.load_translation('events1.json')
events.update(event1_trans)
event1_trans=None

print("Loading events sheet 2/3")
event2_trans = translator.load_translation('events2.json')
events.update(event2_trans)
event2_trans=None

print("Loading events sheet 3/3")
event3_trans = translator.load_translation('events3.json')
events.update(event3_trans)
event3_trans=None

print("Loading qualities")
qualities = translator.load_translation('qualities.json')

Loading events sheet 1/3
Loading events sheet 2/3
Loading events sheet 3/3
Loading qualities


# Check updated items

In [6]:
with open(os.path.join(STATUS_PATH, 'events.json'), 'r') as f:
    last_events = json.load(f)
    
with open(os.path.join(STATUS_PATH, 'qualities.json'), 'r') as f:
    last_qualities = json.load(f)    

In [7]:
event_diff = diff_tree(last_events, events)
qualities_diff = diff_tree(last_qualities, qualities)

if not config['force'] and not event_diff and not qualities_diff:
    log.log("""⁉️ 지자스... 업데이트가 없습니다... 
    따라서 오늘의 패치도 없습니다...😭""") 
    print("No updated")
    sys.exit(0)
    
log.log("""오늘으으은~~! 
events에서 %s개, qualities에서 %s개가 변경 되었습니다! 👍
🔊 처리를 시작합니다 지기지기~ 우우우웅~~ 둠칫둠칫~ 🔊
""" % (event_diff, qualities_diff)) 

오늘으으은~~! 
events에서 0개, qualities에서 0개가 변경 되었습니다! 👍
🔊 처리를 시작합니다 지기지기~ 우우우웅~~ 둠칫둠칫~ 🔊



# Loading origin tree

In [8]:
print("Loading original files")
with open(os.path.join(ORIGIN_PATH, 'events.json'), 'r') as f:
    events_origin = json.load(f)

with open(os.path.join(ORIGIN_PATH, 'qualities.json'), 'r') as f:
    qualities_origin = json.load(f)


Loading original files


# Replacing original text to translated

In [9]:
updater = RecursiveUpdateProcessor()

matched, unmatched = updater.process(events_origin, events)
print('events matched:', matched, 'unmatched:', unmatched, " = ", matched*100/(matched+unmatched), "%")

matched, unmatched = updater.process(qualities_origin, qualities)
print('qualities matched:', matched, 'unmatched:', unmatched, " = ", matched*100/(matched+unmatched), "%")

events matched: 14676 unmatched: 19789  =  42.58232989989845 %
qualities matched: 916 unmatched: 610  =  60.026212319790304 %


In [10]:
print("Saving updated files")
with open(os.path.join(TRANS_PATH, "events.json"), "w") as f:
    json.dump(events_origin, f)
    
with open(os.path.join(TRANS_PATH, "qualities.json"), "w") as f:
    json.dump(qualities_origin, f)

Saving updated files


# Package files

In [ ]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
output_name = "sunless_sea_ko_%s.zip" % now.strftime('%Y%m%d_%H%M')
output_path = os.path.join(OUTPUT_PATH, output_name)

with ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as myzip:
    myzip.write(os.path.join(TRANS_PATH, "events.json"), arcname="events.json")
    myzip.write(os.path.join(TRANS_PATH, "qualities.json"), arcname="qualities.json")
    
print(output_path)
log.log("""

파일 생성 완료! 파일 다운로드는 아래 링크를 이용해 주세요!
--------------------------------------------
http://eggpang.net/sunless/%s
""" % output_name)


# Save last status

In [9]:
with open(os.path.join(STATUS_PATH, 'events.json'), 'w') as f:
    json.dump(events, f)
    
with open(os.path.join(STATUS_PATH, 'qualities.json'), 'w') as f:
    json.dump(qualities, f)    